In [1]:
"""
트리의 앙상블 


-랜덤 포레스트 알고리즘

- 정형 데이터와 비정형 데이터

정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘이 - 앙상블 학습 
이 알고리즘은 대부분 결정 트리를 기반으로 만들어져 있다

비정형 데이터에는 신경망 알고리즘을 주로 사용한다


정형 데이터의 최고인 앙상블 학습 알고리즘
- 랜덤 포레스트
랜덤 포레스트는 앙상블 학습의 대표 주자 중 하나로 안정적인 성능 덕분에 널리 사용되고 있다

부트스트랩 샘플

분류 모델인 RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근 만큼의 특성을 선택
- 즉 4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택하여 사용
다만 회귀 모델인 RandomForestRegressor는 전체 특성을 사용

사이킷런의 RandomForestClassifier 클래스를 화이트 와인을 분류하는 문제에 적용

"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv("https://bit.ly/wine_csv_data")
data = wine[["alcohol", "sugar", "pH"]].to_numpy()
target = wine["class"].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
data, target, test_size=0.2, random_state=42)

In [2]:
"""
cross_validate()함수를 사용해 교차 검증을 수행
RandomForestClassifier

return_train_score 매개변수를 True로 지정하면 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환

결과를 보면 훈련 세트에 다소 과대적합된 모습 - 하지만 예제가 매우 간단하고 특성이 많지 않아 그리드 서치를 사용하더라도
하이퍼파라미터 튜닝의 결과가 크게 나아지지 않는다
"""
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                       return_train_score=True, n_jobs=-1)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9973541965122431 0.8905151032797809


In [3]:
"""
랜덤 포레스트는 결정 트르의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공
criterion, max_depth, max_features, min_samples_split, 등등
결정 트리의 큰 장점 중 하나인 특성 중요도를 계산함
랜덤 포레트스의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것

랜덤 포레스트 모델을 훈련 세트에 훈련한 후 특성 중요도를 출력해 봄
"""
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
"""
RandomForestClassifier에는 자체적으로 모델을 평가하는 점수를 얻을 수 있다
랜덤 포레스트는 훈련 세트에서 중복을 허용하여 부트스트랩 샘플을 만들어 
결정트리를 훈련함
이떄 부트스트랩 샘플에 포함되지 않고 남는 샘플이 있다 - 이런 샘플을 OOB 샘플이라고 함 - 이 남은 샘플을 사용하여
부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있다
oob_score 매개변수를 True로 지정해야함
"""
rf = RandomForestClassifier(oob_score = True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [5]:
"""
엑스트라 트리 
엑스트라 트리는 랜덤 포레스트와 매우 비슷하게 작동함
랜덤 포레스트와 엑스트라 트리의 차이점은 부트스트랩 샘플을 사용하지 않는다는 점
즉 각 결정 트리를 만들 때 전체 훈련 세트를 사용함 
- 대신 노드를 분할할때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할

하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만 
많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있다
사이킷런에서 제공하는 엑스트라 트리는 ExtraTreelassifier
"""
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state =42)
scores = cross_validate(et, train_input, train_target,
                       return_train_score = True, n_jobs=-1)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9974503966084433 0.8887848893166506


In [6]:
"""
랜덤 포레스트와 비슷한 결과를 얻음, 이 예제는 특성이 많지 않아 두 모델의 차이가 크지 않다
보통 엑스트라 트리가 무작위성이 좀 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야함
하지만 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 엑스트라 트리의 장점

엑스트라 트리도 특성 중요도를 제공함
순서는 [알코올 도수, 당도, pH]

엑스트라 트리의 회귀 버젼은 ExtraTreesRegressor 클래스
"""
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [7]:
"""
그레이디언트 부스팅
그레이디어트 부스팅은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 한느 방법
사이킥런의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리를 100개 사용함
깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대함

분류에서는 로지스틱 손실 함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 사용함

GradientBoostingClassifier를 사용해 와인 데이터셋의 교차 검증 점수를 확인

"""
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                       return_train_score = True, n_jobs=-1)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.8881086892152563 0.8720430147331015


In [8]:
"""
과대 적합이 되지 않는 특징
그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강함
학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있다

결정 트리의 개수를 500개로 늘렸지만 과대적합을 잘 억제하고 있음
학습률 learning_rate의 기본값은 0.1임 
"""
gb = GradientBoostingClassifier(n_estimators=500, learning_rate =0.2,
                               random_state=42)
scores= cross_validate(gb, train_input, train_target,
                      return_train_score=True, n_jobs=-1)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9464595437171814 0.8780082549788999


In [9]:
"""
그레이디언트 부스팅도 특성 중요도를 제공함
결과에서 볼 수 있듯이 그레이디언트 부스팅이 랜덤 포레스트보다 일부 특성(당도)에 더 집중
"""
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68011572 0.16116151]


In [10]:
"""
그래디언트 부스팅의 속도와 성능을 개선한 것이 히스토그램 기반 그래디언트 부스팅
- 히스토그램 기반 그레이디언트 부스팅

히스토그램 기반 그레이디언트 부스팅은 정형 데이터를 다루는 머신 러닝 알고리즘 중에 가장 인기가 높은 알고리즘
입력 특성을 256개의 구간으로 나눈다 
따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을수 있다
히스토그램 기반 그레이디언트 부스팅은 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용함
따라서 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없다

사이킷런의 히스토그램 기반 그레이디언트 부스팅 클래스 HistGradientBoostinglassifier
부스팅 반복 횟수를 지정하는 max_iter를 사용함
사이킷런의 히스토그램 기반 그레이디언트 부스팅은 아직 테스트 과정에 있다
이 클래스를 사용하려면 sklearn.experimental 패키지 아래에 있는 enable_hist_gradient_boosting 모듈을 임포트
"""
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

c:\programdata\anaconda3\envs\tensor1\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [11]:
"""
히스토그램 그레이디언트 부스팅의 특성 중요도를 확인
permutation_importance()함수를 사용함

히스토그램 기반 그레이디언트 부스팅 모델을 훈련하고 훈련 세트에서 특성 중요도를 계산해봄
n_repeats 매개변수는 랜덤하게 섞을 횟수를 지정함 - 여기서는 10으로 지정, 기본값은 5

permutation_importance() 함수가 반환하는 객체는 반복하여 얻은 특성 중요도(importances), 
평균(importances_mean), 표준 편차(importances_std)를 담고 있다
"""
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [12]:
"""
테스트 세트에서 특성 중요도를 계산

당도에 집중하고 있다는 것을 확인함
"""
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [13]:
"""
HistGradientBoostingClassifier를 사용해 테스트 세트에서의 성능을 최종적으로 확인
"""
hgb.score(test_input, test_target)

0.8723076923076923

In [17]:
"""
히스토그램 기반 그레이디언트 부스팅 알고리즘을 구현한 라이브러리가 여럿 있다
가장 대표적인 라이브러리는 XGBoost 
사이킷런의 cross_validate() 함수와 함께 사용할 수도 있다 
XGBoost는 다양한 부스팅 알고리즘을 지원함. tree_method 매개변수를 hist로 지정하면
히스토그램 기반 그레이디언트 부스팅을 사용할 수 있다
XGBoost를 사용해 와인 데이터의 교차 검증 점수를 확인
"""
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = "hist", random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

c:\programdata\anaconda3\envs\tensor1\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\programdata\anaconda3\envs\tensor1\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\programdata\anaconda3\envs\tensor1\lib\site-packages\xgboost\sk

[23:07:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:07:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:07:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:07:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

c:\programdata\anaconda3\envs\tensor1\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\programdata\anaconda3\envs\tensor1\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [19]:
"""
널리 사용하는 또 다른 히스토그램 기반 그레이디언트 부스팅 라이브러리는 
마이크로소프트에서 만든 LightGBM임

"""
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs=-1)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))


0.935828414851749 0.8801251203079884
